In [137]:
import json
import sys
sys.path.append(r'C:\Users\udit sharma\Desktop\MSA\2020-Phase-1\Data Science and Machine Learning')
# sys.path.append(r'c:\users\udit sharma\appdata\local\programs\python\python38-32\lib\site-packages')

import pandas as pd
import requests

df = pd.read_csv(r'C:\Users\udit sharma\Desktop\MSA\2020-Phase-1\Data Science and Machine Learning\Dataset.csv')
df_index=pd.read_excel(r'C:\Users\udit sharma\Desktop\MSA\2020-Phase-1\Data Science and Machine Learning\Census Data\otago730395.xlsx')

In [112]:
def get_census(dataframe):
 url='https://koordinates.com/services/query/v1/vector.json'
 census={}
 sa=dataframe.loc[:,'SA1']
 i=0
 for sa1 in sa:


    params = {
        'key' : '7f7a33c9ddc94259bf32bb7ec2df2000',
         'layer' : 104612,
       'x' : dataframe['Longitude'][i],
        'y' : dataframe['Latitude'][i],
        'max_results' : 1

    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print('result issue for:',sa1)
    flag=0
    if(str(response.json()['vectorQuery']['layers']['104612']['features'][0]['properties']['SA12018_V1_00'])==str(sa1)):
        census[sa1]=response.json()['vectorQuery']['layers']['104612']['features'][0]['properties']['C18_CURPop']
        print(sa1)
    else:
        print('The key issue for sa1:',sa1,' Found the sa1:',response.json()['vectorQuery']['layers']['104612']['features'][0]['properties']['SA12018_V1_00'])

    i+=1
 return census       

In [147]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park


In [174]:
census=get_census(df)

In [118]:
deprivation_df.head()

,SA12018_code,NZDep2018,NZDep2018_Score,URPopnSA1_2018,SA22018_code,SA22018_name
0,7000000,10.0,1245.0,141,100100,North Cape
1,7000001,10.0,1245.0,114,100100,North Cape
2,7000002,NaN,NaN,0,100300,Inlets Far North District
3,7000003,10.0,1207.0,225,100100,North Cape
4,7000004,9.0,1093.0,138,100100,North Cape


In [128]:
col1=[]
col2=[]
for key in census:
   col1.append(key)
   col2.append(census[key])    

In [134]:
d={'SA1':col1,'CU18':col2}

In [136]:
import numpy as np
df_census=pd.DataFrame(d)
df_census.head()

,SA1,CU18
0,7009770,174
1,7009991,129
2,7007871,120
3,7008902,231
4,7007917,195


In [162]:
df_depindex=df_index.loc[:,['SA12018_code','NZDep2018_Score','NZDep2018']]
df_depindex.head()

,SA12018_code,NZDep2018_Score,NZDep2018
0,7000000,1245.0,10.0
1,7000001,1245.0,10.0
2,7000002,NaN,NaN
3,7000003,1207.0,10.0
4,7000004,1093.0,9.0


In [163]:

dataframe1=df.merge(df_depindex, left_on='SA1',right_on='SA12018_code',
          suffixes=('none','_right'))

datafinal=dataframe1.merge(df_census, left_on='SA1',right_on='SA1',
          suffixes=('none','_right'))


In [165]:
datafinal=datafinal.drop(columns='SA12018_code')

In [166]:
datafinal.describe()

,Bedrooms,Bathrooms,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,NZDep2018_Score,NZDep2018,CU18
count,1051.000000,1049.000000,1.051000e+03,1051.000000,1051.000000,1.051000e+03,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000,1051.000000
mean,3.777355,2.073403,1.387521e+06,-36.893715,174.799325,7.006319e+06,47.549001,28.963844,27.042816,24.125595,22.615604,29.360609,986.503330,5.063749,179.914367
std,1.169412,0.992985,1.182939e+06,0.130100,0.119538,2.591262e+03,24.692205,21.037441,17.975408,10.942770,10.210578,21.805031,94.287255,2.913471,71.059280
min,1.000000,1.000000,2.700000e+05,-37.265021,174.317078,7.001130e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,849.000000,1.000000,3.000000
25%,3.000000,1.000000,7.800000e+05,-36.950565,174.720779,7.004416e+06,33.000000,15.000000,15.000000,18.000000,15.000000,18.000000,918.000000,2.000000,138.000000
50%,4.000000,2.000000,1.080000e+06,-36.893132,174.798575,7.006325e+06,45.000000,24.000000,24.000000,24.000000,21.000000,27.000000,959.000000,5.000000,174.000000
75%,4.000000,3.000000,1.600000e+06,-36.855789,174.880944,7.008384e+06,57.000000,36.000000,33.000000,30.000000,27.000000,36.000000,1031.000000,8.000000,210.000000
max,17.000000,8.000000,1.800000e+07,-36.177655,175.492424,7.011028e+06,201.000000,270.000000,177.000000,114.000000,90.000000,483.000000,1380.000000,10.000000,789.000000


In [167]:
datafinal.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,NZDep2018_Score,NZDep2018,CU18
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,997.0,6.0,174
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,881.0,1.0,129
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,881.0,1.0,129
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,908.0,2.0,120
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,1091.0,9.0,231


In [168]:
datafinal.dtypes

Bedrooms             int64
Bathrooms          float64
Address             object
Land area           object
CV                   int64
Latitude           float64
Longitude          float64
SA1                  int64
0-19 years           int64
20-29 years          int64
30-39 years          int64
40-49 years          int64
50-59 years          int64
60+ years            int64
Suburbs             object
NZDep2018_Score    float64
NZDep2018          float64
CU18                 int64
dtype: object

In [172]:
datafinal['Land area']=datafinal['Land area'].str.extract('(\d+)').astype('float')

In [173]:
datafinal.dtypes

Bedrooms             int64
Bathrooms          float64
Address             object
Land area          float64
CV                   int64
Latitude           float64
Longitude          float64
SA1                  int64
0-19 years           int64
20-29 years          int64
30-39 years          int64
40-49 years          int64
50-59 years          int64
60+ years            int64
Suburbs             object
NZDep2018_Score    float64
NZDep2018          float64
CU18                 int64
dtype: object